In [1]:
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np
daily_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/BI Argentina/Daily Táctico'
repos_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Bajadas_repo_ventas'
lista_archivos = [x for x in os.listdir(repos_direc) if len(x)==44]

#ver en el OI a que columna corresponde este mes
col_excel_u = 'DQ'
col_excel_vn = 'JT'

In [2]:
[x for x in os.listdir(repos_direc) if len(x)==32]

['VA05_interlocutor_EM_202101.XLSX',
 'VA05_interlocutor_EM_202102.XLSX',
 'VA05_interlocutor_EM_202103.XLSX',
 'VA05_interlocutor_EM_202104.XLSX',
 'VA05_interlocutor_EM_202105.XLSX',
 'VA05_interlocutor_EM_202106.XLSX',
 'VA05_interlocutor_EM_202107.XLSX',
 'VA05_interlocutor_EM_202108.XLSX',
 'VA05_interlocutor_EM_202109.XLSX',
 'VA05_interlocutor_EM_202110.XLSX',
 'VA05_interlocutor_EM_202111.XLSX',
 'VA05_interlocutor_EM_202112.XLSX',
 'VA05_interlocutor_EM_202201.XLSX',
 'VA05_interlocutor_EM_202202.XLSX',
 'VA05_interlocutor_EM_202203.XLSX',
 'VA05_interlocutor_EM_202204.XLSX',
 'VA05_interlocutor_EM_202205.XLSX',
 'VA05_interlocutor_EM_202206.XLSX',
 'VA05_interlocutor_EM_202207.XLSX',
 'VA05_interlocutor_EM_202208.XLSX',
 'VA05_interlocutor_EM_202209.XLSX',
 'VA05_interlocutor_EM_202210.XLSX',
 'VA05_interlocutor_EM_202211.XLSX',
 'VA05_interlocutor_EM_202212.XLSX',
 'VA05_interlocutor_EM_202301.XLSX']

In [3]:
if(date.today().month>9):
    mes = str(date.today().month)
else:
    mes = '0' + str(date.today().month)

In [4]:
#Leo el archivo de materiales
#materiales = pd.read_csv(daily_direc + '/Materiales2.csv', encoding='latin-1', sep = ';')
materiales = pd.read_excel(io = "C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/Tbl_maestro_articulos.xlsx", usecols="D:J,L")
#materiales['Cod_Material'] = materiales['Cod_Material'].astype('int64')

In [5]:
materiales

,Cod_Material,Material,Empresa,Status,BU,Categoría,Marca,Familia
0,8100000282,CICATRICURE CONTORNO OJOS 8.5G LATAM,GLA,DISCO,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE CONTORNO DE OJOS
1,8100001756,ASX JBN FOR CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
2,8100001757,ASX JBN HERBAL CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
3,8100001758,ASX JBN AZF C/ZINC SEBUM X100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
4,8100001759,ASX JBN HUM CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES
...,...,...,...,...,...,...,...,...
914,8110000888,DURACELL PILA LITIO 2016 X1 TIRA,GLA PILAS,BASE,PI,BATTERIES,DURACELL,Pilas litio
915,8110000882,TUKOL MIEL Y TOMILLO 150ML,GLA,INNO 1,OTC,C&C,TUKOL,TUKOL MIEL Y TOMILLO
916,8110000836,CICATRICURE PROMO GOLD NAVIDAD SERUM+DIA,GLA,INNO 1,PC,SKIN CARE,CICATRICURE FACIAL,CICATRICURE GOLD LIFT
917,8110000889,NEXT COMPRIMIDOS GRANEL (SE),NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
%%time

"""este por las dudas lo dejo al parecer funciona bien"""

#quiero armar el acumulado hasta mayo

#me creo un df vacio donde voy acumulando todo
repos_ventas = pd.DataFrame()

#nombres de las columnas del archivo de SAP
SAP_names = ['Doc.fact.', 'Tipo Doc.', 'Folio Entrega', 'Documento de ventas', 'Nombre' ,'Referencia cliente', 
             'Destinatario', 'Material', 'Cantidad facturada', 'Valor neto', 'Importe Bruto', 'Descuentos', 
             'Fecha factura', 'Canal distribución','Organización ventas', 'Clase de factura']

#nombres de las columnas de BD_Venta
BD_names = ['Nro_factura', 'Tipo_doc', 'Nro_entrega', 'Nro_pedido', 'Nbre_Solicitante', 'Nro_OC', 
            'Nro_Destinatario', 'Cod_Material', 'Unidades', 'Valor_Neto', 'Venta_Bruta', 'Descuentos', 
            'Fe_factura', 'Canal_Distribucion', 'Organizacion ventas', 'Clase_de_factura', 'Año_Mes']

#hago un for que me vaya agregando todo hasta mayo
for file in lista_archivos:
    
    #Leo cada archivo y filtro las columnas
    x = pd.read_excel(repos_direc + '/' + file)    
    x = x[SAP_names]
    
    #Agrego la columna Año_Mes usando el nombre del archivo
    x['Año_Mes'] = file[-12:-5].replace('_','-')
    
    #Filtro filas, El material 9100000322 lo pongo cuando tenga los montos
    x = x[x['Organización ventas']=='VNAR']
    x = x[~x['Material'].isin([9100000322, 9100000316])]
    
    #elimino la columna 'Organización ventas' ya que no me sirve mas
    x.drop('Organización ventas', axis=1, inplace=True)
    
    #en el df repos_ventas voy acumulando todos los archivos
    repos_ventas = pd.concat([repos_ventas, x], ignore_index=True)
    
repos_ventas['Tipo'] = 'real'
    
repos_ventas.rename(dict(zip(SAP_names,BD_names)), axis=1, inplace=True)

repos_ventas['Unidades'] = repos_ventas['Unidades'].astype('int64')

dict_replace = {"NC Sin Stock AR" : "NC",
                "Nota de Crédito AR" : "NC",
                "NC Recupero AR" : "NC",
                "Factura Nacional AR" : "Fact",
                "Factura Servicios AR" : "Fact",
                "Nota de Debito AR" : "ND"}

repos_ventas = repos_ventas.replace({"Tipo_doc": dict_replace})

repos_ventas = repos_ventas[repos_ventas['Tipo_doc'].isin(['NC', 'Fact', 'ND'])]

#Me creo el campo Venta Neta
repos_ventas['Valor_Back'] = repos_ventas['Valor_Neto']
repos_ventas.loc[repos_ventas['Clase_de_factura']!='ZNCS', 'Valor_Back'] = 0
#repos_ventas.loc[repos_ventas['Tipo_doc']=='NC', 'Venta_Bruta'] = -repos_ventas['Venta_Bruta']
#repos_ventas.loc[repos_ventas['Tipo_doc']=='NC', 'Descuentos'] = 0
repos_ventas.loc[repos_ventas['Tipo_doc'].isin(['NC', 'ND']), 'Venta_Bruta'] = -repos_ventas['Venta_Bruta']

Wall time: 4min 57s


In [7]:
file

'Reporte de Ventas Nacional vNVA 2023_01.XLSX'

In [8]:
repos_ventas

,Nro_factura,Tipo_doc,Nro_entrega,Nro_pedido,Nbre_Solicitante,Nro_OC,Nro_Destinatario,Cod_Material,Unidades,Valor_Neto,Venta_Bruta,Descuentos,Fe_factura,Canal_Distribucion,Clase_de_factura,Año_Mes,Tipo,Valor_Back
0,2000087316,Fact,170088514.0,170088514,ASOPROFARMA ASOC. PROP.,52000,941,8.110000e+09,560,240561.78,264353.60,-23791.82,2021-01-04,MD,ZFCE,2021-01,real,0.00
1,2000087317,Fact,180522670.0,170088496,DIA ARGENTINA SA,2033324,1899,8.110000e+09,10500,1215043.20,1852200.00,-637156.80,2021-01-04,A1,ZFCE,2021-01,real,0.00
2,2000087317,Fact,180522670.0,170088496,DIA ARGENTINA SA,2033324,1899,8.110000e+09,10500,1215043.20,1852200.00,-637156.80,2021-01-04,A1,ZFCE,2021-01,real,0.00
3,2000087318,Fact,180522671.0,170088498,ALBERDI S.A.,4500320748,760,8.110000e+09,180,39550.50,42075.00,-2524.50,2021-01-04,A2,ZFCE,2021-01,real,0.00
4,2000087318,Fact,180522671.0,170088498,ALBERDI S.A.,4500320748,760,8.110000e+09,264,58007.40,61710.00,-3702.60,2021-01-04,A2,ZFCE,2021-01,real,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674766,2000161121,NC,170166200.0,170166200,ALBERDI S.A.,NCC NOV22 SELLOUT,1100397,8.110001e+09,-1,-2286.18,-2286.18,0.00,2023-01-18,A2,ZNCS,2023-01,real,-2286.18
674767,2000161121,NC,170166200.0,170166200,ALBERDI S.A.,NCC NOV22 SELLOUT,1100397,8.110001e+09,-1,-962.60,-962.60,0.00,2023-01-18,A2,ZNCS,2023-01,real,-962.60
674768,2000161121,NC,170166200.0,170166200,ALBERDI S.A.,NCC NOV22 SELLOUT,1100397,8.110001e+09,-1,-9094.05,-9094.05,0.00,2023-01-18,A2,ZNCS,2023-01,real,-9094.05
674769,2000161121,NC,170166200.0,170166200,ALBERDI S.A.,NCC NOV22 SELLOUT,1100397,8.110001e+09,-1,-8689.87,-8689.87,0.00,2023-01-18,A2,ZNCS,2023-01,real,-8689.87


In [9]:
repos_ventas.groupby(['Año_Mes'])['Unidades'].sum()

Año_Mes
2021-01    2897957
2021-02    2889110
2021-03    4451678
2021-04    3775889
2021-05    3255309
2021-06    3992864
2021-07    3658815
2021-08    4265437
2021-09    4264081
2021-10    3576568
2021-11    3491856
2021-12    3929269
2022-01    4773788
2022-02    6059205
2022-03    6692134
2022-04    6899571
2022-05    6719529
2022-06    6013681
2022-07    6211162
2022-08    6077477
2022-09    5189184
2022-10    4984078
2022-11    6006375
2022-12    4524575
2023-01    3329767
Name: Unidades, dtype: int64

In [10]:
#Son los materiales que estan en repo ventas que no estan en el maestro. El vector deberia tener un len de cero
pd.unique(repos_ventas[~repos_ventas['Cod_Material'].isin(materiales['Cod_Material'])]['Cod_Material']).astype('int64')

array([-9223372036854775808], dtype=int64)

In [11]:
repos2 = pd.merge(repos_ventas,
                materiales,
                left_on = 'Cod_Material',
                right_on = 'Cod_Material',
                how = 'left')

In [12]:
def read_oi(col_excel, col_name):
    
    oi = pd.read_excel(daily_direc + '/OI_CANAL/OI 2022-12 006 CERRADO.XLSX', header = 6, sheet_name = 'Rolling Forecast', 
                   usecols = 'L,' + col_excel)
    
    oi.drop(oi.tail(4).index,inplace=True) # drop last 4 rows

    #agrego la columna de tipo 
    oi['Tipo'] = 'proy'

    #agrego Año_Mes
    oi['Año_Mes'] = datetime.today().strftime('%Y') + '-' + datetime.today().strftime('%m')

    #renombro columnas
    oi_names = ['Cod_Material', col_name, 'Tipo', 'Año_Mes']
    oi.rename(dict(zip(oi.columns.to_list(),oi_names)), axis=1, inplace=True)

    #convierto unidades a int
    oi[col_name] = oi[col_name].fillna(0)
    oi[col_name] = oi[col_name].astype('int64')
    oi = oi.dropna(subset=['Cod_Material'])
    oi['Cod_Material'] = oi['Cod_Material'].astype('int64')

    return oi

In [13]:
oi = pd.merge(read_oi(col_excel_u, 'Unidades'),
                read_oi(col_excel_vn, 'Valor_Neto'),
                left_on = ['Cod_Material', 'Tipo', 'Año_Mes'],
                right_on = ['Cod_Material', 'Tipo', 'Año_Mes'],
                how = 'left')
oi.drop_duplicates(inplace = True)

oi

C:\Users\rvozzi\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


,Cod_Material,Unidades,Tipo,Año_Mes,Valor_Neto
0,8110000329,0,proy,2023-01,0
1,8110000013,0,proy,2023-01,0
2,8110000012,0,proy,2023-01,0
3,8100003447,0,proy,2023-01,0
4,8100003126,0,proy,2023-01,0
...,...,...,...,...,...
2018,8110000895,0,proy,2023-01,0
2019,8110000896,0,proy,2023-01,0
2020,8110000897,0,proy,2023-01,0
2021,8110000775,0,proy,2023-01,0


In [14]:
#Chequeo contra el subtotal del OI de Unidades
oi['Unidades'].sum()

5408734

In [15]:
#Chequeo contra el subtotal del OI de Valor Neto
oi['Valor_Neto'].sum()

3722901750

In [16]:
oi2 = pd.merge(oi, materiales, left_on = 'Cod_Material',
                right_on = 'Cod_Material',
                how = 'inner')

In [17]:
l3m = pd.unique(repos2['Año_Mes'])[-4:-1]
l12m = pd.unique(repos2['Año_Mes'])[-13:-1]

#calculo factores vendidas de l3m
desc_l3m = repos2[repos2['Año_Mes'].isin(l3m)]
desc_l3m = desc_l3m.groupby(['Marca', 'Cod_Material', 'Nbre_Solicitante'],
                    as_index = False)['Unidades'].sum()
desc_l3m.rename(columns={"Unidades": "Unidades3"}, inplace = True)

marca_l3m = repos2[repos2['Año_Mes'].isin(l3m)]
marca_l3m = marca_l3m.groupby(['Marca', 'Cod_Material'],
                    as_index = False)['Unidades'].sum()
marca_l3m.rename(columns={"Unidades": "Total3"}, inplace = True)

factores1 = pd.merge(desc_l3m,
                marca_l3m,
                left_on = ['Marca', 'Cod_Material'],
                right_on = ['Marca', 'Cod_Material'],
                how = 'inner')

factores1['factor1'] = factores1['Unidades3']/factores1['Total3']

#calculo factores vendidas de l12m
desc_l12m = repos2[repos2['Año_Mes'].isin(l12m)]
desc_l12m = desc_l12m.groupby(['Marca', 'Cod_Material', 'Nbre_Solicitante'],
                    as_index = False)['Unidades'].sum()
desc_l12m.rename(columns={"Unidades": "Unidades12"}, inplace = True)

marca_l12m = repos2[repos2['Año_Mes'].isin(l12m)]
marca_l12m = marca_l12m.groupby(['Marca', 'Cod_Material'],
                    as_index = False)['Unidades'].sum()
marca_l12m.rename(columns={"Unidades": "Total12"}, inplace = True)

factores2 = pd.merge(desc_l12m,
                marca_l12m,
                left_on = ['Marca', 'Cod_Material'],
                right_on = ['Marca', 'Cod_Material'],
                how = 'inner')

factores2['factor2'] = factores2['Unidades12']/factores2['Total12']

#junto factores 2 con factores 1 (19239)
factores = pd.merge(factores2,
                   factores1,
                   left_on = ['Marca', 'Cod_Material', 'Nbre_Solicitante'],
                   right_on = ['Marca', 'Cod_Material', 'Nbre_Solicitante'],
                   how = 'left')
factores = factores.fillna(value={'factor1': 0})

factores['factor'] = 0.6*factores['factor1']+0.4*factores['factor2']

In [18]:
factores

,Marca,Cod_Material,Nbre_Solicitante,Unidades12,Total12,factor2,Unidades3,Total3,factor1,factor
0,ASEPXIA,8.100002e+09,JORGE ALBERTO MANSILLA,-36,-36,1.000000,NaN,NaN,0.000000,0.400000
1,ASEPXIA,8.100003e+09,ASOPROFARMA ASOC. PROP.,-2,-72,0.027778,NaN,NaN,0.000000,0.011111
2,ASEPXIA,8.100003e+09,CENTRAL OESTE FARMA A.C.E.,-3,-72,0.041667,NaN,NaN,0.000000,0.016667
3,ASEPXIA,8.100003e+09,DROGUERIA 20 DE JUNIO S.A.,-20,-72,0.277778,NaN,NaN,0.000000,0.111111
4,ASEPXIA,8.100003e+09,DROGUERIA DISVAL S.R.L.,-1,-72,0.013889,-1.0,-40.0,0.025000,0.020556
...,...,...,...,...,...,...,...,...,...,...
20453,X-RAY,8.110001e+09,SALVADO HERMANOS S.A.,240,116087,0.002067,240.0,116087.0,0.002067,0.002067
20454,X-RAY,8.110001e+09,SUIZO ARGENTINA S.A.,23759,116087,0.204665,23759.0,116087.0,0.204665,0.204665
20455,X-RAY,8.110001e+09,UOM BUSTAMANTE S.C.S.,47,116087,0.000405,47.0,116087.0,0.000405,0.000405
20456,X-RAY,8.110001e+09,UOM DELIA SCS,47,116087,0.000405,47.0,116087.0,0.000405,0.000405


In [19]:
res = pd.pivot_table(repos2[repos2['Año_Mes'].isin(l3m)], values='Unidades', index=['Cod_Material', 'Material'],
               #columns='Año_Mes', 
               aggfunc=np.sum)

res = res.reset_index()

agregar = pd.unique(oi2[oi2['Unidades']>0][~oi2[oi2['Unidades']>0]['Cod_Material'].isin(res['Cod_Material'])]['Cod_Material'])

agregar

array([8110000724, 8110000841, 8110000875, 8110000877], dtype=int64)

In [20]:
agregar_val = [8110000701,8110000701, 8110000702, 8110000133, 8110000133]

agregar_factores = pd.DataFrame()

if (len(agregar)==len(agregar_val)):
    #hago un replace de la tabla factores
    #me cargo los df a agregar_factores
    for i in range(0,len(agregar)):
        
        x = factores[factores['Cod_Material']==agregar_val[i]]
        x = x.replace(agregar_val[i], agregar[i])
        
        agregar_factores = pd.concat([agregar_factores, x], ignore_index=True)
        
else:
    print("la lista agregar val debe ser modificada")
    
factores = pd.concat([factores, agregar_factores], ignore_index=True)

la lista agregar val debe ser modificada


In [21]:
factores

,Marca,Cod_Material,Nbre_Solicitante,Unidades12,Total12,factor2,Unidades3,Total3,factor1,factor
0,ASEPXIA,8.100002e+09,JORGE ALBERTO MANSILLA,-36,-36,1.000000,NaN,NaN,0.000000,0.400000
1,ASEPXIA,8.100003e+09,ASOPROFARMA ASOC. PROP.,-2,-72,0.027778,NaN,NaN,0.000000,0.011111
2,ASEPXIA,8.100003e+09,CENTRAL OESTE FARMA A.C.E.,-3,-72,0.041667,NaN,NaN,0.000000,0.016667
3,ASEPXIA,8.100003e+09,DROGUERIA 20 DE JUNIO S.A.,-20,-72,0.277778,NaN,NaN,0.000000,0.111111
4,ASEPXIA,8.100003e+09,DROGUERIA DISVAL S.R.L.,-1,-72,0.013889,-1.0,-40.0,0.025000,0.020556
...,...,...,...,...,...,...,...,...,...,...
20453,X-RAY,8.110001e+09,SALVADO HERMANOS S.A.,240,116087,0.002067,240.0,116087.0,0.002067,0.002067
20454,X-RAY,8.110001e+09,SUIZO ARGENTINA S.A.,23759,116087,0.204665,23759.0,116087.0,0.204665,0.204665
20455,X-RAY,8.110001e+09,UOM BUSTAMANTE S.C.S.,47,116087,0.000405,47.0,116087.0,0.000405,0.000405
20456,X-RAY,8.110001e+09,UOM DELIA SCS,47,116087,0.000405,47.0,116087.0,0.000405,0.000405


In [22]:
oi2[oi2['Cod_Material'].isin([8110000724, 8110000820, 8110000780, 8110000766])]['Unidades'].sum()

22592

In [23]:
tabla = pd.merge(factores[['Marca', 'Cod_Material', 'Nbre_Solicitante', 'factor']], 
                 oi2, 
                 left_on = ['Cod_Material'],
                 right_on = ['Cod_Material'],
                 how = 'inner')
tabla.rename(columns={"Unidades": "Total"}, inplace = True)
tabla.rename(columns={"Valor_Neto": "Total_VN"}, inplace = True)
tabla['Unidades'] = (tabla['factor']*tabla['Total']).apply(np.ceil)
tabla['Unidades'] = tabla['Unidades'].astype('int64')
tabla['Valor_Neto'] = (tabla['factor']*tabla['Total_VN']).apply(np.ceil)
tabla['Valor_Neto'] = tabla['Valor_Neto'].astype('int64')
tabla

,Marca_x,Cod_Material,Nbre_Solicitante,factor,Total,Tipo,Año_Mes,Total_VN,Material,Empresa,Status,BU,Categoría,Marca_y,Familia,Unidades,Valor_Neto
0,ASEPXIA,8.100002e+09,JORGE ALBERTO MANSILLA,0.400000,0,proy,2023-01,0,ASX JBN HUM CON ZINC SEBUM X 100G,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES,0,0
1,ASEPXIA,8.100003e+09,ASOPROFARMA ASOC. PROP.,0.011111,0,proy,2023-01,0,ASEPXIA JABON BARRA CARBON 100G GLOBAL,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES,0,0
2,ASEPXIA,8.100003e+09,CENTRAL OESTE FARMA A.C.E.,0.016667,0,proy,2023-01,0,ASEPXIA JABON BARRA CARBON 100G GLOBAL,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES,0,0
3,ASEPXIA,8.100003e+09,DROGUERIA 20 DE JUNIO S.A.,0.111111,0,proy,2023-01,0,ASEPXIA JABON BARRA CARBON 100G GLOBAL,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES,0,0
4,ASEPXIA,8.100003e+09,DROGUERIA DISVAL S.R.L.,0.020556,0,proy,2023-01,0,ASEPXIA JABON BARRA CARBON 100G GLOBAL,GLA,DISCO,PC,SKIN CARE,ASEPXIA,ASEPXIA JABONES,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20405,X-RAY,8.110001e+09,SALVADO HERMANOS S.A.,0.002067,40000,proy,2023-01,61866058,X-RAY COLAGENO COMPRIMIDOS x 60,GLA,INNO 1,OTC,ANALGESICS,X-RAY,NaN,83,127903
20406,X-RAY,8.110001e+09,SUIZO ARGENTINA S.A.,0.204665,40000,proy,2023-01,61866058,X-RAY COLAGENO COMPRIMIDOS x 60,GLA,INNO 1,OTC,ANALGESICS,X-RAY,NaN,8187,12661846
20407,X-RAY,8.110001e+09,UOM BUSTAMANTE S.C.S.,0.000405,40000,proy,2023-01,61866058,X-RAY COLAGENO COMPRIMIDOS x 60,GLA,INNO 1,OTC,ANALGESICS,X-RAY,NaN,17,25048
20408,X-RAY,8.110001e+09,UOM DELIA SCS,0.000405,40000,proy,2023-01,61866058,X-RAY COLAGENO COMPRIMIDOS x 60,GLA,INNO 1,OTC,ANALGESICS,X-RAY,NaN,17,25048


In [24]:
for ma in pd.unique(oi2['Cod_Material']):
    print(ma)
    print(tabla[tabla['Cod_Material']==ma]['Unidades'].sum())
    print(oi2[oi2['Cod_Material']==ma]['Unidades'].sum())

8110000329
0
0
8110000013
0
0
8110000012
0
0
8100003447
0
0
8100003126
0
0
8110000014
0
0
8110000015
0
0
8110000018
0
0
8100003018
0
0
8110000338
0
0
8110000017
0
0
8100003019
3120
3086
8100001758
0
0
8100003029
0
0
8100001759
0
0
8100001760
0
0
8100001756
0
0
8100001757
0
0
8100001762
0
0
8100001761
0
0
8100003470
9116
9074
8110000624
0
0
8110000777
0
0
8100005576
37414
37372
8100003374
0
0
8110000705
6038
6000
8110000623
0
0
8100002903
0
0
8100003064
0
0
8100003473
0
0
8100003328
19947
19893
8110000706
0
0
8110000625
0
0
8100003022
0
0
8100003466
12722
12689
8110000626
0
0
8100003020
0
0
8100003467
0
0
8110000627
0
0
8100003021
0
0
8110000517
0
0
8110000518
0
0
8110000519
0
0
8110000282
0
0
8110000283
0
0
8110000284
0
0
8110000285
0
0
8110000286
0
0
8110000412
0
0
8110000413
0
0
8110000414
0
0
8110000415
0
0
8110000416
0
0
8110000333
0
0
8110000337
0
0
8110000257
0
0
8110000339
0
0
8110000335
0
0
8110000334
0
0
8110000005
0
0
8110000336
0
0
8110000019
0
0
8110000403
2744
2712
8110000

0
0
8110000710
0
0
8110000711
0
0
8110000091
0
0
8110000089
0
0
8110000086
0
0
8110000087
0
0
8110000088
0
0
8110000082
0
0
8110000083
0
0
8110000084
0
0
8110000085
0
0
8110000090
0
0
8110000081
0
0
8110000155
0
0
8110000154
0
0
8110000156
0
0
8110000157
0
0
8110000158
0
0
8110000159
0
0
8110000160
0
0
8110000125
70995
70970
8110000239
19386
19366
8110000240
20229
20206
8110000280
29286
29263
8110000539
0
0
8110000639
0
0
8110000568
6218
6185
8110000571
9035
9000
8110000657
2828
2800
8110000658
10030
10000
8110000659
1024
1000
8110000572
21156
21123
8110000629
2237
2220
8110000630
11014
11000
8110000631
1518
1500
8110000573
11546
11510
8110000632
2519
2500
8110000633
9524
9500
8110000634
3020
3000
8110000574
0
0
8110000765
0
0
8110000035
0
0
8110000546
33390
33369
8110000133
64037
64012
8110000134
0
0
8110000276
15530
15500
8110000135
0
0
8110000136
8023
8006
8110000137
18366
18345
8100002873
0
0
8110000717
16030
16000
8110000000
0
0
8110000281
0
0
8110000149
442890
442865
8110000150
1

In [25]:
tabla['Unidades'].sum()

5354406

In [26]:
oi['Unidades'].sum()

5408734

In [27]:
tabla['Valor_Neto'].sum()

3688497405

In [28]:
oi['Valor_Neto'].sum()

3722901750

In [29]:
tabla.columns

Index(['Marca_x', 'Cod_Material', 'Nbre_Solicitante', 'factor', 'Total',
       'Tipo', 'Año_Mes', 'Total_VN', 'Material', 'Empresa', 'Status', 'BU',
       'Categoría', 'Marca_y', 'Familia', 'Unidades', 'Valor_Neto'],
      dtype='object')

In [30]:
oi.columns

Index(['Cod_Material', 'Unidades', 'Tipo', 'Año_Mes', 'Valor_Neto'], dtype='object')

In [31]:
oi = tabla[['Cod_Material', 'Nbre_Solicitante','Unidades', 'Valor_Neto', 'Tipo', 'Año_Mes']]

In [32]:
#a estos productos los tengo que agregar a la tabla de materiales
a = oi[~oi['Cod_Material'].isin(materiales['Cod_Material'])]
a[a['Unidades']>0]

,Cod_Material,Nbre_Solicitante,Unidades,Valor_Neto,Tipo,Año_Mes


In [33]:
"""#aca agrego todo lo de G60

gc = pd.read_excel(daily_direc + '/G60/Budget_x_sku_canal.xlsx')

gc = gc[['Cod_Material', 'Unidades', 'Venta Neta contable', 'Año_mes']]

#cambio formato de Año_mes
gc['Año_mes'] = pd.to_datetime(gc['Año_mes']).dt.to_period('M').astype("string")

#renombro las columnas
gc_names = ['Cod_Material', 'Unidades', 'Valor_Neto', 'Año_Mes']
gc.rename(dict(zip(gc.columns.to_list(),gc_names)), axis=1, inplace=True)

#creo la columna tipo
gc['Tipo'] = 'G60'

gc['Unidades'] = gc['Unidades'].astype('int64')"""

'#aca agrego todo lo de G60\n\ngc = pd.read_excel(daily_direc + \'/G60/Budget_x_sku_canal.xlsx\')\n\ngc = gc[[\'Cod_Material\', \'Unidades\', \'Venta Neta contable\', \'Año_mes\']]\n\n#cambio formato de Año_mes\ngc[\'Año_mes\'] = pd.to_datetime(gc[\'Año_mes\']).dt.to_period(\'M\').astype("string")\n\n#renombro las columnas\ngc_names = [\'Cod_Material\', \'Unidades\', \'Valor_Neto\', \'Año_Mes\']\ngc.rename(dict(zip(gc.columns.to_list(),gc_names)), axis=1, inplace=True)\n\n#creo la columna tipo\ngc[\'Tipo\'] = \'G60\'\n\ngc[\'Unidades\'] = gc[\'Unidades\'].astype(\'int64\')'

In [34]:
"""gc"""

'gc'

In [35]:
"""gc.dtypes"""

'gc.dtypes'

In [36]:
base = pd.concat([repos_ventas, oi, 
                  #gc
                 ])

base = pd.merge(base,
                materiales,
                left_on = 'Cod_Material',
                right_on = 'Cod_Material',
                how = 'left')

#junto la base con el catalogo de clientes
catalogo_clientes = pd.read_excel(io = "C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/TBL_canales_SAP_y_gerencias_ventas vR2D2.xlsx")

catalogo_clientes = catalogo_clientes[['Nbre_solicitante', 'Nbre_solicitante2',
       'Canal_distribucion', 'Tipo_estructura5', 'Gerente', 'KAM']]
catalogo_clientes.rename(columns={"Tipo_estructura5": "Canal"}, inplace = True)

base = pd.merge(base,
                catalogo_clientes,
                left_on = 'Nbre_Solicitante',
                right_on = 'Nbre_solicitante',
                how = 'left')

base.loc[base['Nbre_solicitante2'] != 'OTROS', 'Nbre_solicitante'] = base['Nbre_solicitante2']

base.drop(['Nbre_solicitante2'], axis=1, inplace=True)
base.drop(['Nbre_Solicitante'], axis=1, inplace=True)

base['Nro_factura'] = base['Nro_factura'].fillna(0)
base['Nro_factura'] = base['Nro_factura'].astype('int64')

base['Nro_entrega'] = base['Nro_entrega'].fillna(0)
base['Nro_entrega'] = base['Nro_entrega'].astype('int64')

base['Nro_pedido'] = base['Nro_pedido'].fillna(0)
base['Nro_pedido'] = base['Nro_pedido'].astype('int64')

base['Nro_Destinatario'] = base['Nro_Destinatario'].fillna(0)
base['Nro_Destinatario'] = base['Nro_Destinatario'].astype('int64')

base['Valor_Neto'] = base['Valor_Neto'].fillna(0)

base.rename(columns={"Material": "Descripcion"}, inplace = True)

#junto la base con el archivo de ship to x clientes
ship = pd.read_excel('C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/20221018 Detalle de Ship tos X Clientes ARG.XLSX', usecols='AN,AT')
ship.columns = ['ship to', 'ship to poblacion']
ship = ship.drop_duplicates()

base = pd.merge(base,
                ship,
                left_on = 'Nro_Destinatario',
                right_on = 'ship to',
                how = 'left')

In [37]:
base[base['Nbre_solicitante'].isnull()]

,Nro_factura,Tipo_doc,Nro_entrega,Nro_pedido,Nro_OC,Nro_Destinatario,Cod_Material,Unidades,Valor_Neto,Venta_Bruta,...,Categoría,Marca,Familia,Nbre_solicitante,Canal_distribucion,Canal,Gerente,KAM,ship to,ship to poblacion
871,2000087363,Fact,180522678,170088543,7850093462,1901,8.110000e+09,36,6787.99,10892.15,...,HAIR CARE,TIO NACHO,TIO NACHO ENGROSADOR,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
872,2000087363,Fact,180522678,170088543,7850093462,1901,8.110000e+09,42,11878.99,19061.28,...,HAIR CARE,TIO NACHO,TIO NACHO ENGROSADOR,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
873,2000087363,Fact,180522678,170088543,7850093462,1901,8.110000e+09,120,22626.64,36307.19,...,HAIR CARE,TIO NACHO,TIO NACHO ULTRAHIDRATANTE,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
874,2000087363,Fact,180522678,170088543,7850093462,1901,8.110000e+09,102,28848.98,46291.68,...,HAIR CARE,TIO NACHO,TIO NACHO ULTRAHIDRATANTE,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
875,2000087363,Fact,180522678,170088543,7850093462,1901,8.110001e+09,6,1697.00,2723.04,...,HAIR CARE,TIO NACHO,TIO NACHO ALOE VERA,NaN,NaN,NaN,NaN,NaN,1901.0,MORENO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694953,0,NaN,0,0,NaN,0,8.110000e+09,134,110294.00,NaN,...,C&C,TUKOL,TUKOL FORTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
695031,0,NaN,0,0,NaN,0,8.110000e+09,34,29535.00,NaN,...,ANTIMICOTIC,UNESIA,UNESIA UNGÜENTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
695082,0,NaN,0,0,NaN,0,8.110001e+09,42,39296.00,NaN,...,ANTIMICOTIC,UNESIA,PIECIDEX UÑAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
695129,0,NaN,0,0,NaN,0,8.110001e+09,71,175755.00,NaN,...,ANALGESICS,X-RAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
base[base['Tipo']=='proy'].groupby(['Empresa'])['Unidades'].sum()

Empresa
GLA          3683670
GLA PILAS    1670736
Name: Unidades, dtype: int64

In [39]:
base1 = base[base['Año_Mes']=='2022-11']
base1[base1['Tipo']=='real']['Unidades'].sum()

6006375

In [40]:
repos_ventas[repos_ventas['Año_Mes']=='2022-11']['Unidades'].sum()

6006375

In [41]:
#base[base['Tipo']=='G60'].groupby(['Año_Mes'])['Unidades'].sum()

In [42]:
base.to_csv(path_or_buf = daily_direc + '/base_final.csv',
                    sep = ';', index = False)

In [43]:
base

,Nro_factura,Tipo_doc,Nro_entrega,Nro_pedido,Nro_OC,Nro_Destinatario,Cod_Material,Unidades,Valor_Neto,Venta_Bruta,...,Categoría,Marca,Familia,Nbre_solicitante,Canal_distribucion,Canal,Gerente,KAM,ship to,ship to poblacion
0,2000087316,Fact,170088514,170088514,52000,941,8.110000e+09,560,240561.78,264353.6,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE BEAUTY CARE,ASOPROFARMA ASOC. PROP.,MD,1.Farma,Catalina Sabogal,Moira Perusi,941.0,CARAPACHAY
1,2000087317,Fact,180522670,170088496,2033324,1899,8.110000e+09,10500,1215043.20,1852200.0,...,HAIR CARE,TIO NACHO,TIO NACHO HERBOLARIA,DIA ARGENTINA SA,A1,3.Supermercados,Vacante,Nicolas Sardi,1899.0,TORTUGUITAS
2,2000087317,Fact,180522670,170088496,2033324,1899,8.110000e+09,10500,1215043.20,1852200.0,...,HAIR CARE,TIO NACHO,TIO NACHO HERBOLARIA,DIA ARGENTINA SA,A1,3.Supermercados,Vacante,Nicolas Sardi,1899.0,TORTUGUITAS
3,2000087318,Fact,180522671,170088498,4500320748,760,8.110000e+09,180,39550.50,42075.0,...,HAIR CARE,TIO NACHO,TIO NACHO HERBOLARIA,COMODIN,A2,3.Supermercados,Vacante,Sebastian Beltran,760.0,PALPALÁ
4,2000087318,Fact,180522671,170088498,4500320748,760,8.110000e+09,264,58007.40,61710.0,...,HAIR CARE,TIO NACHO,TIO NACHO ACLARANTE,COMODIN,A2,3.Supermercados,Vacante,Sebastian Beltran,760.0,PALPALÁ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695173,0,NaN,0,0,NaN,0,8.110001e+09,83,127903.00,NaN,...,ANALGESICS,X-RAY,NaN,SALVADO HERMANOS S.A.,FR,2.Cadenas farmacias,Catalina Sabogal,David Siciliano,NaN,NaN
695174,0,NaN,0,0,NaN,0,8.110001e+09,8187,12661846.00,NaN,...,ANALGESICS,X-RAY,NaN,SUIZO ARGENTINA S.A.,MD,1.Farma,Catalina Sabogal,Rodrigo Gonzalez Calderon,NaN,NaN
695175,0,NaN,0,0,NaN,0,8.110001e+09,17,25048.00,NaN,...,ANALGESICS,X-RAY,NaN,UOM BUSTAMANTE S.C.S.,FR,2.Cadenas farmacias,Catalina Sabogal,Analia Garcia,NaN,NaN
695176,0,NaN,0,0,NaN,0,8.110001e+09,17,25048.00,NaN,...,ANALGESICS,X-RAY,NaN,UOM DELIA SCS,FR,2.Cadenas farmacias,Catalina Sabogal,Analia Garcia,NaN,NaN


In [44]:
base.columns

Index(['Nro_factura', 'Tipo_doc', 'Nro_entrega', 'Nro_pedido', 'Nro_OC',
       'Nro_Destinatario', 'Cod_Material', 'Unidades', 'Valor_Neto',
       'Venta_Bruta', 'Descuentos', 'Fe_factura', 'Canal_Distribucion',
       'Clase_de_factura', 'Año_Mes', 'Tipo', 'Valor_Back', 'Descripcion',
       'Empresa', 'Status', 'BU', 'Categoría', 'Marca', 'Familia',
       'Nbre_solicitante', 'Canal_distribucion', 'Canal', 'Gerente', 'KAM',
       'ship to', 'ship to poblacion'],
      dtype='object')